In [8]:
import torch
from PIL import Image
import open_clip

#model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
#tokenizer = open_clip.get_tokenizer('ViT-B-32')
model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-H-14')

image = preprocess(Image.open("test.jpg")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat", "stack of english books", "stack of go books"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    print(image_features.shape)
    # Print the data type of the features:
    print(image_features.dtype)

    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

torch.Size([1, 1024])
torch.float32
Label probs: tensor([[3.8599e-09, 6.5063e-11, 1.8181e-10, 1.1857e-03, 9.9881e-01]])


In [7]:
# Process all images in a folder
import os
import numpy as np

image_folder = "images"
image_files = os.listdir(image_folder)
# Get full paths to images

# Get current working directory
cwd = os.getcwd()
print(cwd)
image_files = [os.path.join(cwd, image_folder, f) for f in image_files]

#image_files = [os.path.join(image_folder, f) for f in image_files]
image_files = np.array(image_files)
print(image_files)

batch_size = 4

for i in range(0, max(10, len(image_files)), batch_size):
    batch = image_files[i:i+batch_size]
    print(batch)
    images = [preprocess(Image.open(f)).unsqueeze(0) for f in batch]
    images = torch.cat(images)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model.encode_image(images)
        print(image_features.shape)
        text_features = model.encode_text(text)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

    print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

/mnt/Farscape/Koodi/openclip_db
['/mnt/Farscape/Koodi/openclip_db/images/00007-75_k_lms_3610633945_0.00 (2).jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/2012-07-07 21.47.46.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/20130711_215438.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/20140105_153810.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/20140406_204807.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/20140721_191358.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/20140723_210710.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/2015-01-05 18.11.20.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/2015-02-10 07.51.26.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/2015-03-12 18.45.21.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/2015-08-14 09.06.03.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/2015-08-15 13.53.32.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/2015-10-06 14.56.03.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/2015-11-08 14.15.21.jpg'
 '/mnt/Farscape/Koodi/openclip_db/images/2016-01-